# Code

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import cvxpy as cp
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import warnings
warnings.simplefilter('ignore')
np.random.seed(0)

In [2]:
# First randomly generate a set of 100 flights be used in our calculations
number_of_flights = 100
nodes = 6

# pop is the population of each of cities in our graph; prob will be used for generating our array of flights
pop = np.array([[6.34773e5, 1.97756e5, 3.967e6, 8.74961e5, 2.465e5, 7.24305e5]])
prob = pop / pop.sum()

# Our flights (designated by b)
b = np.zeros(shape=(number_of_flights, nodes))

for i in range(number_of_flights):
    # Generate our b_i by randomly selecting two city pairs with a weighted probability on the more populated cities
    b_ones = np.random.choice(nodes, size=2, replace=False, p=np.squeeze(prob))
    b[i, b_ones[0]] = 1 # Pick a random starting city for flight
    b[i, b_ones[1]] = -1 # Pick a random end city for flight

# d is the cost of going through an airport, which is a function of population size as well (we'll say a factor of 0.001)
d = 0.001*pop.T

print('---First Ten Flights Generated---')
print(b[0:10])
print('\nKey:\n-1 = Departure city\n 1 = Arrival City')

---First Ten Flights Generated---
[[ 0.  0.  1. -1.  0.  0.]
 [ 0.  0.  1.  0. -1.  0.]
 [ 0.  0.  1.  0.  0. -1.]
 [ 0.  0. -1.  0.  0.  1.]
 [ 0.  0. -1.  1.  0.  0.]
 [ 0.  0.  1.  0.  0. -1.]
 [ 1. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.]
 [ 0.  0.  0. -1.  0.  1.]
 [ 0.  0.  1. -1.  0.  0.]]

Key:
-1 = Departure city
 1 = Arrival City


## Hub-and-spoke Model

### Primal Problem

In [3]:
# Set the number of edges for the given model (Hub-and-spoke)
edges = 10

# Costs are equivalent to the number of minutes required to travel between two cities (nodes) of our graph
c = np.array([[65, 65, 85, 85, 70, 60, 150, 140, 80, 75]]).T

# Generate our E matrices based on the hub-and-spoke routes
E_i = np.array([[1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]])
E_o = np.array([[0, 1, 0, 1, 0, 1, 0, 1,  0, 1],
                [0,  0, 0,  0, 0,  0, 0,  0, 1,  0],
                [1, 0, 0,  0, 0,  0, 0,  0,  0,  0],
                [0,  0, 1, 0, 0,  0, 0,  0,  0,  0],
                [0,  0,  0, 0,1,  0, 0,  0,  0,  0],
                [0,  0,  0, 0, 0,  0,1,  0,  0,  0]])

E = E_i - E_o

# x is a variable with shape |e| x |i| where i is the the number of routes
x = cp.Variable([edges, number_of_flights])
obj = 0.
constraints = [x>=0]

for i in range(number_of_flights):
    obj = obj + (c.T + d.T@E_o)@x[:, i]

    constraints.append(E @ x[:, i] == b[i, :])

primal = cp.Problem(cp.Minimize(obj), constraints)
primal.solve()

print('Primal Problem')
print('Optimal Value:', np.round_((primal.value / 60), decimals=2))

print('\nOptimal x (i.e. Route Taken for Given Flight):')
print(np.round_(np.absolute(x.value[:, 0:5]), decimals=1))
print('\nKey:\n Cols = Flight(s)\n Rows = Edges Traveled (i.e. Legs of Flight)\n 1 = City is in route')

Primal Problem
Optimal Value: 3303.8

Optimal x (i.e. Route Taken for Given Flight):
[[0. 0. 0. 1. 1.]
 [1. 1. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

Key:
 Cols = Flight(s)
 Rows = Edges Traveled (i.e. Legs of Flight)
 1 = City is in route


### Dual Problem

In [4]:
v = cp.Variable([nodes, 1])
mu = cp.Variable([edges, 1])

obj = cp.sum(-v.T @ b.T)
constraints = [mu >= 0]
constraints.append((c.T + d.T@E_o) + v.T@E - mu.T == 0)

dual = cp.Problem(cp.Maximize(obj), constraints)
dual.solve()

print('Dual Problem')
print('Optimal Value:', np.round_((dual.value / 60), decimals=2))

print(v.value)

print(np.round_(v.value, decimals=3))
print(np.round_(mu.value, decimals=1))

Dual Problem
Optimal Value: 857.97
[[-288.12483322]
 [ -10.36883448]
 [-987.89783316]
 [ 671.83616632]
 [  28.37516554]
 [ 586.18016636]]
[[-288.125]
 [ -10.369]
 [-987.898]
 [ 671.836]
 [  28.375]
 [ 586.18 ]]
[[4731.8]
 [   0. ]
 [   0. ]
 [1679.7]
 [   0. ]
 [1011.3]
 [   0. ]
 [1649.1]
 [   0. ]
 [ 987.5]]


## Point-to-point Model

### Primal Problem

In [5]:
# Costs are equivalent to the number of minutes required to travel between two cities (nodes) of our graph
c = np.array([[65, 65, 80, 80, 70, 80, 130, 115, 125, 125, 110, 100, 75, 80, 60, 75]]).T

# Generate our E matrices based on the hub-and-spoke routes
E_i = np.array([[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
                [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]])

E_o = np.array([[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]])

E = E_i - E_o

# x is a variable with shape |e| x |i| where i is the the number of routes
edges=16
x = cp.Variable([edges, number_of_flights])
obj = 0.
constraints = [x>=0]

for i in range(number_of_flights):
    obj = obj + (c.T + d.T@E_o)@x[:, i]

    constraints.append(E @ x[:, i] == b[i,:])

primal = cp.Problem(cp.Minimize(obj), constraints)
primal.solve()

print('Primal Solution:', np.round_((primal.value / 60), decimals=2))

print('\n---Route Taken for Given Flight---')
print(np.round_(np.absolute(x.value[:, 0:5]), decimals=1))
print('\nKey:\n Cols = Flight(s)\n Rows = Edges Traveled (i.e. Legs of Flight)\n 1 = City is in route')

Primal Solution: 3035.53

---Route Taken for Given Flight---
[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

Key:
 Cols = Flight(s)
 Rows = Edges Traveled (i.e. Legs of Flight)
 1 = City is in route


In [6]:
v = cp.Variable([nodes, 1])
mu = cp.Variable([edges, 1])

obj = cp.sum(-v.T @ b.T)
constraints = [mu >= 0]
constraints.append((c.T + d.T@E_o) + v.T@E - mu.T == 0)

dual = cp.Problem(cp.Maximize(obj), constraints)
dual.solve()

print('Dual Problem')
print('Optimal Value:', np.round_((dual.value / 60), decimals=2))

print(v.value)

print(np.round_(v.value, decimals=3))
print(np.round_(mu.value, decimals=1))

Dual Problem
Optimal Value: 802.36
[[-209.02733294]
 [  68.72866478]
 [-908.80033261]
 [  46.16066717]
 [ 107.4726648 ]
 [ 895.46566661]]
[[-209.027]
 [  68.729]
 [-908.8  ]
 [  46.161]
 [ 107.473]
 [ 895.466]]
[[   0. ]
 [4731.8]
 [   0. ]
 [5002. ]
 [   0. ]
 [1031.3]
 [1154.5]
 [  12.6]
 [1849.3]
 [   0. ]
 [1144.5]
 [  36.3]
 [ 987.5]
 [   0. ]
 [ 996.3]
 [ 260.2]]
